In [1]:
import torch
print('GPU not found.') if not torch.cuda.is_available() else print(f'Found GPU: {torch.cuda.get_device_name(0)}')

Found GPU: NVIDIA GeForce RTX 3070


In [2]:
# Cell 1: 라이브러리 import
import torch
from utils.cnn.classifier import initialize_cnn_classifier
from utils.cnn.dataset_functions import create_cnn_dataset, unwrap_client_data, get_label_mapping
from utils.cnn.federated_averaging import federated_averaging
from utils.cnn.visualization import plot_training_curves, visualize_predictions

In [ ]:
# Cell 2: 데이터 경로 및 클라이언트 분배 설정 (수정 버전)
import random
import numpy as np

data_dir = 'data'
imagePath0 = f'{data_dir}/0/'
imagePath1 = f'{data_dir}/1/'
npyPath = f'{data_dir}/annotations/'

# 파일 분배 설정
total_files = 693
num_clients = 6
test_client_id = 6  # 테스트용 클라이언트 ID
test_data_ratio = 0.10  # 각 클라이언트에서 추출할 비율 (10%)

# 전체 파일 리스트 생성
all_files = [f'{i:06d}' for i in range(1, total_files + 1)]
random.shuffle(all_files)  # 랜덤 셔플 (선택사항)

# client6을 제외한 클라이언트에 데이터 분배
train_clients = [f'client{i}' for i in range(1, num_clients + 1) if i != test_client_id]
num_train_clients = len(train_clients)
files_per_train_client = len(all_files) // num_train_clients

clientIdentifierDict = {}
test_files = []  # client6에 들어갈 파일들

# 각 학습 클라이언트에 데이터 분배 및 테스트 데이터 추출
start_idx = 0
for i, client_id in enumerate(train_clients):
    if i < num_train_clients - 1:
        end_idx = start_idx + files_per_train_client
    else:
        end_idx = len(all_files)
    
    # 클라이언트에 할당된 파일들
    client_files = all_files[start_idx:end_idx]
    
    # 테스트 데이터 추출 (각 클라이언트에서 일정 비율)
    num_test_from_client = int(len(client_files) * test_data_ratio)
    test_files_from_client = random.sample(client_files, num_test_from_client)
    test_files.extend(test_files_from_client)
    
    # 남은 파일들을 클라이언트에 할당
    train_files = [f for f in client_files if f not in test_files_from_client]
    clientIdentifierDict[client_id] = train_files
    
    print(f'{client_id}: {len(train_files)}개 파일 (테스트로 {num_test_from_client}개 추출)')
    start_idx = end_idx

# client6에 테스트 파일 할당
clientIdentifierDict[f'client{test_client_id}'] = test_files
print(f'\nclient{test_client_id} (테스트): {len(test_files)}개 파일 (다양한 클라이언트에서 추출)')
print(f'총 학습 데이터: {sum(len(files) for k, files in clientIdentifierDict.items() if k != f"client{test_client_id}")}개')
print(f'총 테스트 데이터: {len(test_files)}개')

client1: 85개 파일 (테스트로 14개 추출)
client2: 85개 파일 (테스트로 14개 추출)
client3: 85개 파일 (테스트로 14개 추출)
client4: 85개 파일 (테스트로 14개 추출)
client5: 85개 파일 (테스트로 14개 추출)
client7: 85개 파일 (테스트로 14개 추출)
client8: 85개 파일 (테스트로 14개 추출)

client6 (테스트): 98개 파일 (다양한 클라이언트에서 추출)
총 학습 데이터: 595개
총 테스트 데이터: 98개


In [4]:
# Cell 3: 레이블 매핑 생성
label_mapping, num_classes = get_label_mapping(data_dir, clientIdentifierDict)

발견된 결함 유형 (0과 1 제외): [np.int8(-1), np.uint8(3), np.uint8(4), np.uint8(5), np.uint8(6), np.uint8(7), np.uint8(8), np.uint8(9), np.uint8(11), np.uint8(14), np.uint8(255)]
레이블 매핑: {np.int8(-1): 0, np.uint8(3): 1, np.uint8(4): 2, np.uint8(5): 3, np.uint8(6): 4, np.uint8(7): 5, np.uint8(8): 6, np.uint8(9): 7, np.uint8(11): 8, np.uint8(14): 9, np.uint8(255): 10}
총 클래스 수: 11


In [5]:
# # 결함 유형 확인
# from utils.cnn.visual_npy import visualize_defect_types_samples

# visualize_defect_types_samples(data_dir, clientIdentifierDict, label_mapping, min_pixels=150)

In [6]:
# Cell 4: 데이터셋 생성
target_size = (640, 640)
print("\n데이터셋 생성 중...")
imageDict, labelDict = create_cnn_dataset(
    clientIdentifierDict,
    data_dir,
    target_size=target_size,
    label_mapping=label_mapping
)


데이터셋 생성 중...
client1...
  처리 중: 50/85 파일 완료
Contains 85 images...
Valid images: 78 (filtered 7 invalid labels)
Image Tensor Shape: torch.Size([78, 2, 640, 640])
Label Shape: torch.Size([78])
Label distribution: Counter({np.int64(6): 33, np.int64(1): 14, np.int64(4): 12, np.int64(3): 8, np.int64(7): 4, np.int64(10): 3, np.int64(5): 2, np.int64(0): 1, np.int64(8): 1})
client2...
  처리 중: 50/85 파일 완료
Contains 85 images...
Valid images: 80 (filtered 5 invalid labels)
Image Tensor Shape: torch.Size([80, 2, 640, 640])
Label Shape: torch.Size([80])
Label distribution: Counter({np.int64(6): 24, np.int64(4): 20, np.int64(1): 16, np.int64(3): 10, np.int64(5): 4, np.int64(7): 3, np.int64(0): 2, np.int64(10): 1})
client3...
  처리 중: 50/85 파일 완료
Contains 85 images...
Valid images: 83 (filtered 2 invalid labels)
Image Tensor Shape: torch.Size([83, 2, 640, 640])
Label Shape: torch.Size([83])
Label distribution: Counter({np.int64(6): 27, np.int64(1): 20, np.int64(4): 14, np.int64(3): 10, np.int64(10): 

In [7]:
# Cell 5: Train/Test 분할
trainClients = ['client1', 'client2', 'client3', 'client4', 
                'client5', 'client7', 'client8']
testClients = ['client6']

clientIDs = trainClients

# Train Data
trainImageDict = {clientID: imageDict[clientID] for clientID in trainClients}
trainLabelDict = {clientID: labelDict[clientID] for clientID in trainClients}

# Test Data
testImageDict = {clientID: imageDict[clientID] for clientID in testClients}
testLabelDict = {clientID: labelDict[clientID] for clientID in testClients}

testImages, testLabels = unwrap_client_data(testImageDict, testLabelDict, testClients)


In [8]:
# # 레이블 검증
# print("\n레이블 범위 검증:")
# for clientID in clientIDs:
#     labels = trainLabelDict[clientID]
#     min_label = labels.min().item()
#     max_label = labels.max().item()
#     invalid_count = ((labels < 0) | (labels >= num_classes)).sum().item()
#     print(f"{clientID}: min={min_label}, max={max_label}, invalid={invalid_count}")
#     if invalid_count > 0:
#         print(f"  경고: {clientID}에 유효하지 않은 레이블이 {invalid_count}개 있습니다!")

In [9]:
# Cell 6: 하이퍼파라미터 설정
SERVER_ROUNDS = 10
LOCAL_EPOCHS = 15
LOCAL_BATCH_SIZE = 32
LOCAL_LEARNING_RATE = 1e-4

In [10]:
# Cell 7: 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = initialize_cnn_classifier(num_classes, input_channels=2, device=device)
print(f"모델 초기화 완료! 클래스 수: {num_classes}")


모델 초기화 완료! 클래스 수: 11


In [11]:
# Cell 8: 연합학습 실행
model, serverStateDict, lossDict, testLoss, accuracyDict, testAccuracy = federated_averaging(
    model,
    SERVER_ROUNDS, LOCAL_EPOCHS, LOCAL_BATCH_SIZE, LOCAL_LEARNING_RATE,
    clientIDs, trainImageDict, trainLabelDict,
    testImages, testLabels,
    num_classes,
    device=device
)

------ Server Round 0 ------

Running local updates for client1...
Epoch 1/15 - Loss: 2.3337, Accuracy: 30.77%
Epoch 2/15 - Loss: 2.1278, Accuracy: 64.10%
Epoch 3/15 - Loss: 1.9177, Accuracy: 65.38%
Epoch 4/15 - Loss: 1.7672, Accuracy: 75.64%
Epoch 5/15 - Loss: 1.5873, Accuracy: 70.51%
Epoch 6/15 - Loss: 1.3972, Accuracy: 75.64%
Epoch 7/15 - Loss: 1.2842, Accuracy: 69.23%
Epoch 8/15 - Loss: 1.4208, Accuracy: 66.67%
Epoch 9/15 - Loss: 1.3047, Accuracy: 74.36%
Epoch 10/15 - Loss: 0.9493, Accuracy: 79.49%
Epoch 11/15 - Loss: 1.0354, Accuracy: 74.36%
Epoch 12/15 - Loss: 0.8518, Accuracy: 74.36%
Epoch 13/15 - Loss: 0.8583, Accuracy: 78.21%
Epoch 14/15 - Loss: 0.8505, Accuracy: 85.90%
Epoch 15/15 - Loss: 0.7618, Accuracy: 82.05%
Saving local updates for client1...

Running local updates for client2...
Epoch 1/15 - Loss: 2.3425, Accuracy: 22.50%
Epoch 2/15 - Loss: 2.2254, Accuracy: 43.75%
Epoch 3/15 - Loss: 1.9699, Accuracy: 65.00%
Epoch 4/15 - Loss: 1.8732, Accuracy: 71.25%
Epoch 5/15 - Loss

KeyboardInterrupt: 

In [ ]:
# Cell 9: 학습 곡선 시각화
plot_training_curves(lossDict, accuracyDict, testLoss, testAccuracy, clientIDs)


In [ ]:
# Cell 10: 모델 저장
import os
from utils.cnn.classifier import save_cnn_model

os.makedirs('saved_models', exist_ok=True)

test_set_name = ''.join([c.replace('client', '') for c in testClients])
if len(test_set_name) == 1:
    test_set_name = '0' + test_set_name

lr_str = f"{LOCAL_LEARNING_RATE:.0e}".replace('-', '').replace('+', '').replace('.0', '')
base_filename = f'saved_models/CNN_FL_{SERVER_ROUNDS}_{LOCAL_EPOCHS}_{LOCAL_BATCH_SIZE}_{lr_str}_HoldoutPart{test_set_name}.pth'

model_filename = base_filename
counter = 1
while os.path.exists(model_filename):
    base_name, ext = os.path.splitext(base_filename)
    model_filename = f'{base_name}_{counter}{ext}'
    counter += 1

save_cnn_model(model, model_filename, num_classes=num_classes)
print(f'\n최종 테스트 성능:')
print(f'  Loss: {testLoss[-1]:.4f}')
print(f'  Accuracy: {testAccuracy[-1]:.2f}%')


In [ ]:
# Cell 11: 예측 결과 시각화
visualize_predictions(
    model, testImageDict, testLabelDict, testClients,
    clientIdentifierDict, data_dir, num_samples=6, device=device
)